In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max.columns', None)

from scipy.stats import skew
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score,auc

from sklearn.neighbors import KNeighborsClassifier

In [2]:
dataset = pd.read_csv("pd_speech_features.csv")

dataset = dataset.drop(['id'],axis=1)
dataset.shape
dataset.head()

,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,GNE_SNR_SEO,GNE_NSR_TKEO,GNE_NSR_SEO,VFER_mean,VFER_std,VFER_entropy,VFER_SNR_TKEO,VFER_SNR_SEO,VFER_NSR_TKEO,VFER_NSR_SEO,IMF_SNR_SEO,IMF_SNR_TKEO,IMF_SNR_entropy,IMF_NSR_SEO,IMF_NSR_TKEO,IMF_NSR_entropy,mean_Log_energy,mean_MFCC_0th_coef,mean_MFCC_1st_coef,mean_MFCC_2nd_coef,mean_MFCC_3rd_coef,mean_MFCC_4th_coef,mean_MFCC_5th_coef,mean_MFCC_6th_coef,mean_MFCC_7th_coef,mean_MFCC_8th_coef,mean_MFCC_9th_coef,mean_MFCC_10th_coef,mean_MFCC_11th_coef,mean_MFCC_12th_coef,mean_delta_log_energy,mean_0th_delta,mean_1st_delta,mean_2nd_delta,mean_3rd_delta,mean_4th_delta,mean_5th_delta,mean_6th_delta,mean_7th_delta,mean_8th_delta,mean_9th_delta,mean_10th_delta,mean_11th_delta,mean_12th_delta,mean_delta_delta_log_energy,mean_delta_delta_0th,mean_1st_delta_delta,mean_2nd_delta_delta,mean_3rd_delta_delta,mean_4th_delta_delta,mean_5th_delta_delta,mean_6th_delta_delta,mean_7th_delta_delta,mean_8th_delta_delta,mean_9th_delta_delta,mean_10th_delta_delta,mean_11th_delta_delta,mean_12th_delta_delta,std_Log_energy,std_MFCC_0th_coef,std_MFCC_1st_coef,std_MFCC_2nd_coef,std_MFCC_3rd_coef,std_MFCC_4th_coef,std_MFCC_5th_coef,std_MFCC_6th_coef,std_MFCC_7th_coef,std_MFCC_8th_coef,std_MFCC_9th_coef,std_MFCC_10th_coef,std_MFCC_11th_coef,std_MFCC_12th_coef,std_delta_log_energy,std_0th_delta,std_1st_delta,std_2nd_delta,std_3rd_delta,std_4th_delta,std_5th_delta,std_6th_delta,std_7th_delta,std_8th_delta,std_9th_delta,std_10th_delta,std_11th_delta,std_12th_delta,std_delta_delta_log_energy,std_delta_delta_0th,std_1st_delta_delta,std_2nd_delta_delta,std_3rd_delta_delta,std_4th_delta_delta,std_5th_delta_delta,std_6th_delta_delta,std_7th_delta_delta,std_8th_delta_delta,std_9th_delta_delta,std_10th_delta_delta,std_11th_delta_delta,std_12th_delta_delta,Ea,Ed_1_coef,Ed_2_coef,Ed_3_coef,Ed_4_coef,Ed_5_coef,Ed_6_coef,Ed_7_coef,Ed_8_coef,Ed_9_coef,Ed_10_coef,det_entropy_shannon_1_coef,det_entropy_shannon_2_coef,det_entropy_shannon_3_coef,det_entropy_shannon_4_coef,det_entropy_shannon_5_coef,det_entropy_shannon_6_coef,det_entropy_shannon_7_coef,det_entropy_shannon_8_coef,det_entropy_shannon_9_coef,det_entropy_shannon_10_coef,det_entropy_log_1_coef,det_entropy_log_2_coef,det_entropy_log_3_coef,det_entropy_log_4_coef,det_entropy_log_5_coef,det_entropy_log_6_coef,det_entropy_log_7_coef,det_entropy_log_8_coef,det_entropy_log_9_coef,det_entropy_log_10_coef,det_TKEO_mean_1_coef,det_TKEO_mean_2_coef,det_TKEO_mean_3_coef,det_TKEO_mean_4_coef,det_TKEO_mean_5_coef,det_TKEO_mean_6_coef,det_TKEO_mean_7_coef,det_TKEO_mean_8_coef,det_TKEO_mean_9_coef,det_TKEO_mean_10_coef,det_TKEO_std_1_coef,det_TKEO_std_2_coef,det_TKEO_std_3_coef,det_TKEO_std_4_coef,det_TKEO_std_5_coef,det_TKEO_std_6_coef,det_TKEO_std_7_coef,det_TKEO_std_8_coef,det_TKEO_std_9_coef,det_TKEO_std_10_coef,app_entropy_shannon_1_coef,app_entropy_shannon_2_coef,app_entropy_shannon_3_coef,app_entropy_shannon_4_coef,app_entropy_shannon_5_coef,app_entropy_shannon_6_coef,app_entropy_shannon_7_coef,app_entropy_shannon_8_coef,app_entropy_shannon_9_coef,app_entropy_shannon_10_coef,app_entropy_log_1_coef,app_entropy_log_2_coef,app_entropy_log_3_coef,app_entropy_log_4_coef,app_entropy_log_5_coef,app_entropy_log_6_coef,app_entropy_log_7_coef,app_entropy_log_8_coef,app_entropy_log_9_coef,app_entropy_log_10_coef,app_det_TKEO_mean_1_coef,app_det_TKEO_mean_2_coef,app_det_TKEO_mean_3_coef,app_det_TKEO_mean_4_coef,app_det_TKEO_mean_5_coef,app_det_TKEO_mean_6_coef,app_det_TKEO_mean_7_coef,app_det_TKEO_mean_8_coef,app_det_TKEO_mean_9_coef,app_det_TKEO_mean_10_coef,app_TKEO_std_1_coef,app_TKEO_std_2_coef,app_TKEO_std

In [3]:
column_names = list(dataset.columns)
column_names = column_names[:-1] # Removing the last attribute which is the 'CLASS'
print(column_names)

['gender', 'PPE', 'DFA', 'RPDE', 'numPulses', 'numPeriodsPulses', 'meanPeriodPulses', 'stdDevPeriodPulses', 'locPctJitter', 'locAbsJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter', 'locShimmer', 'locDbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'meanAutoCorrHarmonicity', 'meanNoiseToHarmHarmonicity', 'meanHarmToNoiseHarmonicity', 'minIntensity', 'maxIntensity', 'meanIntensity', 'f1', 'f2', 'f3', 'f4', 'b1', 'b2', 'b3', 'b4', 'GQ_prc5_95', 'GQ_std_cycle_open', 'GQ_std_cycle_closed', 'GNE_mean', 'GNE_std', 'GNE_SNR_TKEO', 'GNE_SNR_SEO', 'GNE_NSR_TKEO', 'GNE_NSR_SEO', 'VFER_mean', 'VFER_std', 'VFER_entropy', 'VFER_SNR_TKEO', 'VFER_SNR_SEO', 'VFER_NSR_TKEO', 'VFER_NSR_SEO', 'IMF_SNR_SEO', 'IMF_SNR_TKEO', 'IMF_SNR_entropy', 'IMF_NSR_SEO', 'IMF_NSR_TKEO', 'IMF_NSR_entropy', 'mean_Log_energy', 'mean_MFCC_0th_coef', 'mean_MFCC_1st_coef', 'mean_MFCC_2nd_coef', 'mean_MFCC_3rd_coef', 'mean_MFCC_4th_coef', 'mean_MFCC_5th_coef', 'mean_MFCC_6th_coef', 'mean_MFCC_7th_coef', '

<h1>SKEWNESS REDUCTION</h1>

In [4]:
skewedCols = []
for names in column_names:
    skewVal = dataset[names].skew()
    if(skewVal>1 or skewVal<-1):
        skewedCols.append(names)
# print(skewedCols)
print(len(skewedCols))

526


In [5]:
#   Name of the cols containing atleast one negative value
skewedCols_NegativeVals = []
for col_name in skewedCols:
    for values in dataset[col_name]:
        if(values < 0):
            skewedCols_NegativeVals.append(col_name)
            break
print(len(skewedCols_NegativeVals))

#   Name of the cols containing atleast one zero values
skewedCols_ZeroVals = []
for col_name in skewedCols:
    if col_name in skewedCols_NegativeVals:
        continue
    else:
        for values in dataset[col_name]:
            if(values == 0):
                skewedCols_ZeroVals.append(col_name)
                break
print(len(skewedCols_ZeroVals))

#   Name of the cols containing only positive values
skewedCols_PositiveVals = []
for col_name in skewedCols:
    if (col_name not in skewedCols_NegativeVals and col_name not in skewedCols_ZeroVals):
        skewedCols_PositiveVals.append(col_name)
print(len(skewedCols_PositiveVals))

147
1
378


In [6]:
#   Box-Cox Transformation
for col_name in skewedCols_PositiveVals:
    dataset[col_name] = stats.boxcox(dataset[col_name],lmbda=0)

count = 0
for col_name in skewedCols_PositiveVals:
    skewVal = dataset[col_name].skew()
    if skewVal>1 or skewVal<-1:
        count = count + 1
print(count)
print(dataset)

168
     gender       PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0         1 -0.159617  0.71826  0.57227        240               239   
1         1 -0.265451  0.69481  0.53966        234               233   
2         1 -0.161543  0.67604  0.58982        232               231   
3         0 -0.888651  0.79672  0.59257        178               177   
4         0 -1.115047  0.79782  0.53028        236               235   
..      ...       ...      ...      ...        ...               ...   
751       0 -0.211919  0.56355  0.28385        417               416   
752       0 -1.827345  0.56499  0.59194        415               413   
753       0 -0.123423  0.72335  0.46815        381               380   
754       0 -0.176952  0.74890  0.49823        340               339   
755       0 -0.206975  0.76471  0.46374        340               339   

     meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0            0.008064           -9.351904     -6.128430   

In [7]:
#   CubeRoot Transformation on skewedCols_NegativeVals & skewedCols_ZeroVals
for col_name in skewedCols_NegativeVals:
    dataset[col_name] = np.cbrt(dataset[col_name])
for col_name in skewedCols_ZeroVals:
    dataset[col_name] = np.cbrt(dataset[col_name])

count = 0
for col_name in skewedCols_NegativeVals:
    skewVal = dataset[col_name].skew()
    if skewVal>1 or skewVal<-1:
        count = count + 1
print(count)
print(dataset)

48
     gender       PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0         1 -0.159617  0.71826  0.57227        240               239   
1         1 -0.265451  0.69481  0.53966        234               233   
2         1 -0.161543  0.67604  0.58982        232               231   
3         0 -0.888651  0.79672  0.59257        178               177   
4         0 -1.115047  0.79782  0.53028        236               235   
..      ...       ...      ...      ...        ...               ...   
751       0 -0.211919  0.56355  0.28385        417               416   
752       0 -1.827345  0.56499  0.59194        415               413   
753       0 -0.123423  0.72335  0.46815        381               380   
754       0 -0.176952  0.74890  0.49823        340               339   
755       0 -0.206975  0.76471  0.46374        340               339   

     meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0            0.008064           -9.351904     -6.128430    

In [8]:
# #   Log Transformation on skewedCols_PositiveVals
# for col_name in skewedCols_PositiveVals:
#     dataset[col_name] = np.log(dataset[col_name])

# count = 0
# for col_name in skewedCols_PositiveVals:
#     skewVal = dataset[col_name].skew()
#     if skewVal>1 or skewVal<-1:
#         count = count + 1
# print(count)

<h1>Kurtosis Reduction</h1>

In [9]:
kurtosisCols = []
for names in column_names:
    kurtVal = dataset[names].kurt()
    if(kurtVal>3 or kurtVal<3):
        kurtosisCols.append(names)
# print(skewedCols)
print(len(kurtosisCols))

753


<h1>Outlier Detection</h1>

In [10]:
import statistics
for col_name in column_names:
    qi = dataset[col_name].quantile(0.25)
    qf = dataset[col_name].quantile(0.75)

    iqr = qf - qi
    c = 1.5
    upper_limit=qf+c*iqr
    lower_limit=qi-c*iqr

    col_median = statistics.mean(dataset[col_name].to_numpy())
    for val in dataset[col_name]:
        if val < lower_limit or val > upper_limit:
            dataset[col_name] = dataset[col_name].replace(val,col_median)

In [11]:
print(dataset)

     gender       PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0         1 -0.159617  0.71826  0.57227        240               239   
1         1 -0.265451  0.69481  0.53966        234               233   
2         1 -0.161543  0.67604  0.58982        232               231   
3         0 -0.341503  0.79672  0.59257        178               177   
4         0 -0.341503  0.79782  0.53028        236               235   
..      ...       ...      ...      ...        ...               ...   
751       0 -0.211919  0.56355  0.28385        417               416   
752       0 -0.341503  0.56499  0.59194        415               413   
753       0 -0.123423  0.72335  0.46815        381               380   
754       0 -0.176952  0.74890  0.49823        340               339   
755       0 -0.206975  0.76471  0.46374        340               339   

     meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0            0.008064           -9.351904     -6.128430    -10

<h1>Testing the Dataset with a Model</h1>

In [12]:
print(dataset.shape)

standard = StandardScaler()

real_x = dataset.iloc[:,:-1].values
real_y = dataset.iloc[:,-1].values

real_x = standard.fit_transform(real_x)

x_train, x_test, y_train, y_test = train_test_split(real_x, real_y, test_size=0.1)

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)   
print(knn.score(x_test, y_test))
y_predict_knn = knn.predict(x_test)
cm_knn = confusion_matrix(y_test, y_predict_knn)

(756, 754)
0.8947368421052632


Standard scaling.

In [13]:
from sklearn.preprocessing import StandardScaler
# get the features and label from the original dataframe
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]
# performing standardization
sc = StandardScaler()
X_scaled = sc.fit_transform(X)


In [23]:
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset

df = dataset
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[['class']])
output=transformed.toarray()

# print(transformed.toarray())

In [26]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
shape = list(dataset.shape)
# for i in range(11,55):
components = 17
_sc = StandardScaler()
_pca = PCA(n_components = components)

_model = MLPClassifier(hidden_layer_sizes=(shape[0]//2,shape[1]//2))
pipeline_model = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
    ('classifier', _model)
])
# perform a split
X_train, X_test, y_train, y_test =train_test_split(X, output, test_size=0.1, random_state=0)
# train the model using the PCA components
pipeline_model.fit(X_train,y_train)

print(pipeline_model.score(X_test,y_test))

0.9210526315789473
